In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os
from lips import get_root_path
# from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
from lips.dataset.airfransDataSet import AirfRANSDataSet
import numpy as np

In [2]:
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'Dataset'
BENCHMARK_NAME = "DEFAULT"
LOG_PATH = LIPS_PATH + "lips_logs.log"
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations","benchmarks","confAirfoil.ini") 

directory_name='Dataset'

Similar model to the one in MLP_test_model.ipynb. However, here we have 128 nodes in hidden layer and using ReLU instead Tanh.

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 4)
        )
    
    def forward(self, x):
        return self.model(x)

In [4]:
def train_model(model, train, test, epochs=50):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    device = torch.device("cpu")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # model.eval()
        # val_loss = 0.0
        # with torch.no_grad():
        #     for X_val, y_val in test:
        #         X_val, y_val = X_val.to(device), y_val.to(device)
        #         y_pred = model(X_val)
        #         loss = criterion(y_pred, y_val)
        #         val_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss/len(train):.4f}")
        # print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss/len(test):.4f} | Test Loss: {val_loss/len(test):.4f}")
    
    return model

In [16]:
def load_data(dataset):
    # Deifinition of Input and Output attributes
    X_attr = ["x-position", "y-position", "x-inlet_velocity", "y-inlet_velocity", "x-normals", "y-normals", "distance_function"]
    y_attr = ["x-velocity", "y-velocity", "pressure", "turbulent_viscosity"]
    X, y = [], []
    
    # Looping through 10 items from the dataset to get the attributes
    for i in range(1,35907642):
        X.append(np.column_stack([dataset.data[attr][i] for attr in X_attr]))
        y.append(np.column_stack([dataset.data[attr][i] for attr in y_attr]))

    # Stacking the data to form a single matrix
    X = np.vstack(X)
    y = np.vstack(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
    
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    
    return train_loader, test_loader

In [6]:
attr_names = (
        'x-position',
        'y-position',
        'x-inlet_velocity', 
        'y-inlet_velocity', 
        'distance_function', 
        'x-normals', 
        'y-normals', 
        'x-velocity', 
        'y-velocity', 
        'pressure', 
        'turbulent_viscosity',
    )
attr_x = attr_names[:7]
attr_y = attr_names[7:]

Taken from 2-Import_Airfoil_design_Dataset.ipynb

In [7]:
configuration_file = None #Convenient alternative but not required at this point
dataset_name = "my_dataset_test"
usecase_task = "scarce" #Four task are supported: 'full', 'scarce', 'reynolds', 'aoa'
usecase_split = "training" #Describe which data subset within a task to be used, the other option is testing
log_path = "dataset_log"

In [8]:
my_dataset = AirfRANSDataSet(config = configuration_file, 
                             name = dataset_name,
                             task = usecase_task,
                             split = usecase_split,
                             attr_names = attr_names, 
                             attr_x = attr_x, 
                             attr_y = attr_y, 
                             log_path = log_path)

In [9]:
my_dataset.load(path=DIRECTORY_NAME)

Loading dataset (task: scarce, split: train): 100%|██████████| 200/200 [00:36<00:00,  5.47it/s]


In [15]:
my_dataset.data['x-position'].shape

(35907642,)

In [17]:
train, test = load_data(my_dataset) # Took around 8 minutes

In [18]:
model = MLP()

In [19]:
trained_model = train_model(model, train, test) # 220 min

Epoch 1/50 | Loss: 564570.6549
Epoch 2/50 | Loss: 365984.4185
Epoch 3/50 | Loss: 322295.1194
Epoch 4/50 | Loss: 287690.9939
Epoch 5/50 | Loss: 249313.2268
Epoch 6/50 | Loss: 214248.8200
Epoch 7/50 | Loss: 191472.1132
Epoch 8/50 | Loss: 174811.0045
Epoch 9/50 | Loss: 169234.3456
Epoch 10/50 | Loss: 150391.4979
Epoch 11/50 | Loss: 146287.5184
Epoch 12/50 | Loss: 136238.8990
Epoch 13/50 | Loss: 126794.0700
Epoch 14/50 | Loss: 120513.0955
Epoch 15/50 | Loss: 117777.5674
Epoch 16/50 | Loss: 113314.9937
Epoch 17/50 | Loss: 108756.0943
Epoch 18/50 | Loss: 104055.8228
Epoch 19/50 | Loss: 102635.6302
Epoch 20/50 | Loss: 99981.7662
Epoch 21/50 | Loss: 95522.7067
Epoch 22/50 | Loss: 95588.2278
Epoch 23/50 | Loss: 94317.6919
Epoch 24/50 | Loss: 92798.3962
Epoch 25/50 | Loss: 89484.8576
Epoch 26/50 | Loss: 85487.8752
Epoch 27/50 | Loss: 86601.1200
Epoch 28/50 | Loss: 86663.4564
Epoch 29/50 | Loss: 86594.6364
Epoch 30/50 | Loss: 78555.1687
Epoch 31/50 | Loss: 80814.3462
Epoch 32/50 | Loss: 83696.990